## Software dependencies

- Troppo package (Github "dev" branch: https://github.com/BioSystemsUM/troppo/tree/dev)
- cobamp package (Github "dev" branch: https://github.com/BioSystemsUM/cobamp/tree/dev)
- CPLEX solver 12.9 (requires academic license: https://www.ibm.com/products/ilog-cplex-optimization-studio/pricing)
    - Does not work with Python versions >= 3.8. Please check the compatiblity between the installed Python versions and the corresponding CPLEX solvers.
- "reconstruction.py" example code (Github "CODE" subfolder: https://github.com/MariaPessoa/thesis_annexes/tree/main/CODE)
- cobrapy, pandas & numpy packages


NOTE: As context-specific model extraction can be a lengthy process, using a computer cluster is recommended. 

## Extraction input files and parameters

Model extraction refers to the process of the extraction of a context-specific model (CSM) from a generic template model by a reconstruction algorithm, based on omics data. The Troppo package’s model extraction pipeline requires a template model, gene scores or gene expression data and an optional protected reaction set, such as reactions required by essential metabolic tasks, as direct input data. Additionally, Troppo requires the selection of the appropriate activity threshold, of the algorithm(s) with which to extract the CSMs and the gene mapping method. 
<p>In particular, the Troppo package’s implementation of (t)INIT does not accept a task file to determine which reactions to prioritize, but instead accepts a protected reaction set, similarly to fastCORE’s core reactions. Furthermore, the template model should not have blocked reactions, which can lead to extraction issues with fastCORE. After extraction, the pipeline outputs the reaction content of all CSMs. 

- gene-level TPM by tissue data (GTEx Project (v8) median TPM data: https://gtexportal.org/home/datasets)
- generic template model (Human-GEM (v1.5, SBML/XML format): https://github.com/SysBioChalmers/Human-GEM/tree/v1.5.0)

### Pre-processing

The typical extraction process involves gene score thresholding strategy selection and subsequent conversion of gene expression data into gene scores. However, in this quick tutorial gene expression data pre-processing will follow the method used by Robinson et al. (2020), which simply sets a global activity threshold of 1 TPM, without any conversion process.
<p>Likewise, although the inclusion of reactions required by essential tasks would lead to higher quality models, only a minimal set of protected reactions related to biomass will be considered. 

Github documentation: https://sysbiochalmers.github.io/Human-GEM-guide/gem_extraction/

In [25]:
import os
from cobra.io import read_sbml_model,write_sbml_model
from cobra.flux_analysis import find_blocked_reactions
import pandas as pd

ROOT_FOLDER = "C:\\Users\\liamo\\Documents\\BIOINF\\PRETHESIS"
model_path = os.path.join(ROOT_FOLDER,"Human-GEM","model","Human-GEM.xml")
template_path = os.path.join(ROOT_FOLDER,"Human-GEM","model","Human-GEM_ct.xml")
data_path = os.path.join(ROOT_FOLDER,"DATA","GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_median_tpm.gct")
output_path = os.path.join(ROOT_FOLDER,"DATA","results","gtl50_gtu90_lt50_fttinit_mediantpm4_csm_nodrugex.csv")

#### Reference code for model pre-processing:

As using cobrapy's "find_blocked_reactions" function may cause the "TypeError: can't pickle SwigPyObject objects" in Windows, this example will leave the code for reference but directly import the pre-processed template model instead. 

In [12]:
model = read_sbml_model(model_path)

In [13]:
print("Genes:",len(model.genes))
print("Reactions:",len(model.reactions))
print("Metabolites:",len(model.metabolites))

Genes: 3626
Reactions: 13096
Metabolites: 8400


In [ ]:
blocked = find_blocked_reactions(model) 
model.remove_reactions(blocked) #may require "remove_orphans=True" depending on version 
write_sbml_model(model,'Human-GEM_ct.xml')

#### Template model

In [14]:
template = read_sbml_model(template_path)

In [15]:
print("Genes:",len(template.genes))
print("Reactions:",len(template.reactions))
print("Metabolites:",len(template.metabolites))

Genes: 3052
Reactions: 11902
Metabolites: 7140


#### Gene expression data

The "reconstruction.py" script is expecting data with ENSEMBL IDs, in the format of samples x genes. 

In [18]:
data_path = os.path.join(ROOT_FOLDER,"DATA","GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_median_tpm.gct")
data = pd.read_csv(data_path,header=0,skiprows=[0,1],sep="\t",index_col="Name").drop(["Description"],axis=1)

In [24]:
data.shape

(56200, 54)

In [20]:
data.iloc[:5,:5]

,Adipose - Subcutaneous,Adipose - Visceral (Omentum),Adrenal Gland,Artery - Aorta,Artery - Coronary
Name,,,,,
ENSG00000223972.5,0.00000,0.00000,0.00000,0.00000,0.00000
ENSG00000227232.5,4.06403,3.37111,2.68549,4.04762,3.90076
ENSG00000278267.1,0.00000,0.00000,0.00000,0.00000,0.00000
ENSG00000243485.5,0.00000,0.00000,0.00000,0.00000,0.00000
ENSG00000237613.2,0.00000,0.00000,0.00000,0.00000,0.00000


#### Example extraction parameters for use in reconstruction.py

- Reconstruction algorithm(s): fastcore
- Activity threshold: 1 (TPM)
- Gene mapping: MINMAX
- protected_reactions = ["biomass_human", "HMR_10023", "HMR_10024"]

In [ ]:
params = {'algorithms':['fastcore'],
    'strategies': {'tinit': integration_strategy_map['adjusted_score'](protected_reactions),
                  'fastcore': integration_strategy_map['default_core'](1, protected_reactions)}, 
    'functions':{'minmax': MINMAX}}

#### Example output

In [34]:
med = pd.read_csv(output_path,header=0,index_col=1)
med.iloc[:,:5]

,Unnamed: 0,Unnamed: 2,HMR_3905,HMR_3907,HMR_4097
Adipose - Subcutaneous,tinit,minmax,True,True,True
Breast - Mammary Tissue,tinit,minmax,True,True,True
Kidney - Cortex,tinit,minmax,True,True,True
Liver,tinit,minmax,True,True,True
Adipose - Subcutaneous,fastcore,minmax,True,True,True
Breast - Mammary Tissue,fastcore,minmax,True,True,True
Kidney - Cortex,fastcore,minmax,True,True,True
Liver,fastcore,minmax,True,True,True


As all models were extracted with the MINMAX gene mapping method, this column can be dropped on import. 

In [35]:
med = pd.read_csv(output_path,header=0,index_col=1).drop(["Unnamed: 2"], axis=1)

Fastcore models only:

In [ ]:
med.index = pd.MultiIndex.from_tuples([tuple(x.split("_")) for x in med.index])
med_ft = med.loc[med["Unnamed: 0"]=="fastcore",:].drop(["Unnamed: 0"], axis=1)

In [28]:
med_ft.shape

(4, 11386)

In [29]:
med_ft.iloc[:,:5]

,HMR_3905,HMR_3907,HMR_4097,HMR_4099,HMR_4108
Adipose - Subcutaneous,True,True,True,False,True
Breast - Mammary Tissue,True,True,True,False,True
Kidney - Cortex,True,True,True,True,True
Liver,True,True,True,False,True


In [33]:
med_ft.sum(axis=1)

Adipose - Subcutaneous     6498
Breast - Mammary Tissue    6544
Kidney - Cortex            5244
Liver                      6497
dtype: int64

To obtain individual model objects that can be exported to SBML format, for example, load the template model and remove all reactions not present in the CSM.

In [ ]:
all_reactions = set([r.id for r in template.reactions])

for i,row in med_ft.iterrows(): #expects reaction content
    # using with statements to change the COBRA model temporarily
    # this is done to knock-out reaction not appearing in the FASTCORE result
    with template as context_specific_model: 
        to_remove = row[(row==False)].index.to_list()
        for r in to_remove: 
            # if r in all_reactions:
            context_specific_model.reactions.get_by_id(r).knock_out() # knock-out reactions not in the model